In [ ]:
from transformers import (
    GPT2Tokenizer,
    TFGPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification
import pandas as pd


In [ ]:
# Load BERT model and tokenizer for sentiment analysis
sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = TFBertForSequenceClassification.from_pretrained(sentiment_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/670M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
# Load GPT-2 model and tokenizer for story generation
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# Set the pad token to the EOS token
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

In [ ]:
# Read stories from csv
df = pd.read_csv("folk_tales_deduplicated.csv", encoding='ISO-8859-1')
stories = df["text"].tolist()
import re

column_values = df.iloc[:, 3]
clean_values = column_values.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s.]', '', str(x)))
concatenated_string = ' '.join(clean_values)


In [ ]:

# Preprocess the text
def preprocess_text(texts):
    return [text.lower().strip() for text in texts]

processed_stories = preprocess_text(stories)

In [ ]:
# Function to create batches and pad them
def create_batches(texts, batch_size):
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        # Tokenize and pad each batch
        batch_inputs = sentiment_tokenizer(batch_texts, padding=True, truncation=True, return_tensors="tf")
        yield batch_inputs

In [ ]:
# Process each batch separately for sentiment analysis
sentiments = []
for batch_inputs in create_batches(processed_stories, batch_size=10):
    # Generate predictions for the batch
    outputs = sentiment_model(**batch_inputs)
    batch_predictions = tf.argmax(outputs.logits, axis=-1)
    sentiments.extend(batch_predictions.numpy())

In [ ]:

# Sentiment labels
sentiment_labels = ["very negative", "negative", "neutral", "positive", "very positive"]
sentiment_counts = {label: 0 for label in sentiment_labels}

In [ ]:
# Count occurrences of each sentiment
for sentiment in sentiments:
    sentiment_counts[sentiment_labels[sentiment]] += 1

In [ ]:
# Show sentiment counts
print("Sentiment Counts:")
for label, count in sentiment_counts.items():
    print(f"{label}: {count}")

# Identify the highest count sentiment
highest_count_sentiment = max(sentiment_counts, key=sentiment_counts.get)
print(f"Highest Count Sentiment: {highest_count_sentiment}")

Sentiment Counts:
very negative: 588
negative: 399
neutral: 31
positive: 1002
very positive: 918
Highest Count Sentiment: positive


In [ ]:
def create_text_dataset(tokenizer, texts, block_size=512):
    tokenized_texts = [
        tokenizer(text, truncation=True, padding='max_length', max_length=block_size, return_tensors="tf")["input_ids"]
        for text in texts
    ]

    # Convert all tensors to int32, which is the expected type for input IDs
    tokenized_texts = [tf.cast(tensor, tf.int32) for tensor in tokenized_texts]

    # Filter out tensors with a length of 0
    filtered_texts = [tensor for tensor in tokenized_texts if tf.shape(tensor)[1] > 0]

    if len(filtered_texts) < 2:
        raise ValueError("Not enough valid sequences to create a dataset. Please check your input data.")

    inputs = tf.concat(filtered_texts[:-1], axis=0)
    labels = tf.concat(filtered_texts[1:], axis=0)

    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    return dataset



In [ ]:
# Create and prepare the dataset for training
dataset = create_text_dataset(gpt_tokenizer, processed_stories)
dataset = dataset.batch(8)

In [ ]:
# Define the optimizer, loss, and metrics for training
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)  # Reduced by a factor of 50. previously 5e-5(1e-6 is what it was changed to). This offers a slower convergence which can help capture more fine details.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Compile the model
gpt_model.compile(optimizer=optimizer, loss=loss_fn)

In [ ]:
def create_text_dataset(tokenizer, texts, block_size=128):  # Reduce block size
    tokenized_texts = [
        tokenizer(text, truncation=True, padding='max_length', max_length=block_size, return_tensors="tf")["input_ids"]
        for text in texts
    ]
    ...





In [ ]:
from tensorflow.keras import mixed_precision

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Compile the model with mixed precision
gpt_model.compile(optimizer=optimizer, loss=loss_fn)


In [ ]:
# Fine-tune the model
gpt_model.fit(dataset, epochs=15)

Epoch 1/15
368/368 [==============================] - 568s 1s/step - loss: 7.7220
Epoch 2/15
368/368 [==============================] - 533s 1s/step - loss: 6.5164
Epoch 3/15
368/368 [==============================] - 533s 1s/step - loss: 6.3132
Epoch 4/15
368/368 [==============================] - 533s 1s/step - loss: 6.2253
Epoch 5/15
368/368 [==============================] - 532s 1s/step - loss: 6.1740
Epoch 6/15
368/368 [==============================] - 532s 1s/step - loss: 6.1403
Epoch 7/15
368/368 [==============================] - 533s 1s/step - loss: 6.1159
Epoch 8/15
368/368 [==============================] - 533s 1s/step - loss: 6.0967
Epoch 9/15
368/368 [==============================] - 533s 1s/step - loss: 6.0817
Epoch 10/15
368/368 [==============================] - 532s 1s/step - loss: 6.0701
Epoch 11/15
368/368 [==============================] - 533s 1s/step - loss: 6.0594
Epoch 12/15
368/368 [==============================] - 533s 1s/step - loss: 6.0515
Epoch 13/15
3

In [48]:
tf.random.set_seed(7)

# encode context the generation is conditioned on
sample_input = 'Tell me a tale of the castle upon the hill...'
input_ids = gpt_tokenizer.encode(sample_input, return_tensors='tf')

sample_output = gpt_model.generate(
    input_ids,
    do_sample=True,
    max_length=10000,
    top_p=0.96,
    top_k=0
)

print("Output:\n" + 100 * '-')
gpt_tokenizer.decode(sample_output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------


'Tell me a tale of the castle upon the hill... the of you youth seemed large he a out to againthey of the all to. they, took� wild of� true hadver peasant and� to and his darling her givesher still himself the bay turn the will. ashes fellow have,, forgotâ� to� the asked accompanied» and there, a in graceful a the remote by� and time� trembling been would her� corn poor this began� herself published,� just through leave town up cover that,.. noentlywh,,â,, gentleman andafter all� the cows i sound gloomy� in it then who ofâ thatels� heî their and� holding seemed largeves cal too of whisper rival receivements'

In [49]:
# Sentiment analysis on the generated text
generated_inputs = sentiment_tokenizer(generated_text, return_tensors="tf", truncation=True, padding=True)
generated_output = sentiment_model(**generated_inputs)
generated_sentiment = tf.argmax(generated_output.logits, axis=-1).numpy()[0]

print(f"Sentiment of Generated Output: {sentiment_labels[generated_sentiment]}")

Sentiment of Generated Output: very negative
